In [16]:
import Data as Data
from keras.layers import Reshape, merge, Input, Dense, Flatten
from keras.engine import  Model
from keras_vggface import VGGFace
from scipy.spatial.distance import cosine
import numpy as np
import scipy.misc

In [27]:
def data_init(notop = False):

    print('Data loading..')
    (X_1, X_2, Y )= Data.loadData('/home/danning/lfw/lfw_data.h5')
    #len = 6000
    print(X_1.shape)

    
    if notop:
        X_1 = (X_1.astype(np.float32) - 127.5)/127.5
        X_2 = (X_2.astype(np.float32) - 127.5)/127.5
        return (X_1,X_2,Y)
    else:
        print('Imgae risizing...')
        #image array after resize
        Xr_1=[]
        Xr_2=[]

        for img in X_1:
            Xr_1.append(scipy.misc.imresize(img, size=(224,224,3),interp='nearest'))
        for img in X_2:
            Xr_2.append(scipy.misc.imresize(img, size=(224,224,3),interp='nearest'))

        Xr_1 = np.asarray(Xr_1)
        Xr_2 = np.asarray(Xr_2)
    
        #print('Data Loaded. Now normalizing..')
        Xr_1 = (Xr_1.astype(np.float32) - 127.5)/127.5
        Xr_2 = (Xr_2.astype(np.float32) - 127.5)/127.5
        return (Xr_1,Xr_2,Y)
    

In [28]:
def vgg_model(notop = False):
    if notop:
        vgg_model = VGGFace(include_top=False, input_shape=(112,96, 3), pooling='avg') # pooling: None, avg or max
        
    else:
        vgg_model = VGGFace(include_top=True, input_shape=(224, 224, 3), pooling='avg') # pooling: None, avg or max
        out = vgg_model.get_layer('fc7').output
        vgg_model = Model(vgg_model.input, out)
    vgg_model.summary()
    return vgg_model

In [29]:
def verification(model,data):
    print('Feature vectors extracting...')

    (X_1,X_2,Y) = data
    fvec1=model.predict(X_1, verbose=0)
    fvec2=model.predict(X_2, verbose=0)
    
    print('Cosine similarity calculating...')
    cos_sim = []
    for i in range(len(fvec1)):
        cos_sim.append(1-cosine(fvec1[i],fvec2[i]))
    #print(cos_sim)

    print('cos_sim range: {0},{1}'.format(min(cos_sim),max(cos_sim)))
    
    
    f = [] #f_measurement
    max_f = 0 
    max_f_index = -1 
    pre = [] #precision
    rec = [] #recall
    for i in range(len(cos_sim)):
        threshold = cos_sim[i]
        tp=0 #true positive
        fp=0 #false positive
        fn=0 #false negative
        for j in range(len(cos_sim)):
            if cos_sim[j]>=threshold:
                if Y[j] == 1:
                    tp+=1
                if Y[j] == 0:
                    fp+=1
            else:
                if Y[j] == 1:
                    fn+=1
        pre.append(tp/(tp+fp))
        rec.append(tp/(tp+fn))
        temp = 2*pre[i]*rec[i]/(pre[i]+rec[i])
        f.append(temp)
        if temp>max_f:
            max_f = temp
            max_f_index = i

    threshold = cos_sim[max_f_index]
    print('threshold:{0}'.format(threshold))
    
    
    y_test = []
    err=0
    for i in range(len(cos_sim)):
        y_test.append(1 if cos_sim[i]>=threshold else 0) 
        if Y[i]!=y_test[i]:
            err+=1

    acc=1-err/len(Xr_1)
    print(acc)

In [24]:
#notop : 88.7%
#with top : 95.4%
data = data_init(notop=True)
model = vgg_model(notop=True)
verification(model,data)

Data loading..
(6000, 112, 96, 3)
Imgae risizing...
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         (None, 112, 96, 3)        0         
_________________________________________________________________
conv1_1 (Conv2D)             (None, 112, 96, 64)       1792      
_________________________________________________________________
conv1_2 (Conv2D)             (None, 112, 96, 64)       36928     
_________________________________________________________________
pool1 (MaxPooling2D)         (None, 56, 48, 64)        0         
_________________________________________________________________
conv2_1 (Conv2D)             (None, 56, 48, 128)       73856     
_________________________________________________________________
conv2_2 (Conv2D)             (None, 56, 48, 128)       147584    
_________________________________________________________________
pool2 (MaxPooling2D)    

In [30]:
data = data_init(notop=False)
model = vgg_model(notop=False)
verification(model,data)

Data loading..
(6000, 112, 96, 3)
Imgae risizing...
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
conv1_1 (Conv2D)             (None, 224, 224, 64)      1792      
_________________________________________________________________
conv1_2 (Conv2D)             (None, 224, 224, 64)      36928     
_________________________________________________________________
pool1 (MaxPooling2D)         (None, 112, 112, 64)      0         
_________________________________________________________________
conv2_1 (Conv2D)             (None, 112, 112, 128)     73856     
_________________________________________________________________
conv2_2 (Conv2D)             (None, 112, 112, 128)     147584    
_________________________________________________________________
pool2 (MaxPooling2D)    